<h4> Importando as bibliotecas

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

<h4> Abrindo o dataset

In [4]:
#Carregando o dataset 
df = pd.read_csv("Newdata.csv")

In [5]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churn,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,DIAMOND,456
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,DIAMOND,377
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,GOLD,350
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,GOLD,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0,DIAMOND,300
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,PLATINUM,771
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1,SILVER,564
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,GOLD,339


<h2> Feature engineering, split e pré-processamento

Para obter a melhor performance possível dos modelos de testes, é necessário realizar algumas transformações e criações de features que impulsionem o rendimento dos algoritmos. Abaixo explico melhor as features e as transformações do pré-processador que vou desenvolver

Pré-processador:

Como apenas modelos baseados em árvores serão utilizados, o pré-processador terá:
- Nenhuma transformação em colunas numéricas, pois modelos de árvores não precisam que os dados estejam na mesma escala para performarem bem.
- Para preservar a característica ordinal da coluna "Card Type", será aplicado Ordinal Encoding
- Para as colunas "Geography" e "Gender", será aplicado One-Hot Encoding. Isso funcionará bem pois há poucos valores únicos nessa coluna (2 em Gender e 3 em Geography), assim o dataset expandirá pouco e não afetará a qualidade do modelo.

Feature engineering:  

Afim de extrair o máximo de informações relevantes possíveis do dataset e melhorar o desempenho do algoritmo, algumas features serão criadas. Todas as features criadas serão determinísticas e não causarão data leakage, mesmo sendo realizadas antes do split dos dados. As features são:
- Engagement (HasCrCard + IsActiveMember + NumOfProducts)
- Financial_Value (Balance + EstimatedSalary)
- Age_per_Tenure (Age / (Tenure + 0.1))
- Balance_per_Tenure (Balance / (Tenure + 1))
- Products_per_Tenure (NumOfProducts / (Tenure + 1))
- Points_per_Tenure (Point Earned / (Tenure + 1))
- FinancialValue_per_Tenure (Financial_Value / (Tenure + 1))
- Engagement_per_Age (Engagement / Age)
- Balance_per_Products (Balance / NumOfProducts)
- Older_Inactive (Cliente com mais de 50 anos e inativo)
- Low_value_Churner (Clientes inativos, com balanço igual a 0 e com 0 ou 1 produtos adquiridos)

<h3> 1. Feature engineering

In [6]:
#Cópia do dataset carregado
df_train = df.copy()

#Criação das features
df_train["Engagement"] = (df_train['HasCrCard'] + df_train['IsActiveMember'] + df_train['NumOfProducts'])
df_train["Financial_value"] = (df_train['Balance'] + df_train['EstimatedSalary'])
df_train['Older_Inactive'] = ((df_train['Age'] >= 50) & (df_train['IsActiveMember'] == 0)).astype(int)
df_train['Low_value_Churner'] = ((df_train['IsActiveMember'] == 0) &(df_train['Balance'] == 0) &(df_train['NumOfProducts'] <= 1)).astype(int)
df_train['Age_per_Tenure'] = (df_train['Age'] / (df_train['Tenure'] + 0.1)).astype('float64')
df_train['Balance_per_Tenure'] = (df_train['Balance'] / (df_train['Tenure'] + 0.1)).astype('float64')
df_train['Products_per_Tenure'] = (df_train['NumOfProducts'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['Points_per_Tenure'] = (df_train['Point Earned'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['FinancialValue_per_Tenure'] = (df_train['Financial_value'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['Engagement_per_Age'] = (df_train['Engagement'] / (df_train['Age'])).astype('float64')
df_train['Balance_per_Products'] = (df_train['Balance'] / (df_train['NumOfProducts'])).astype('float64')

In [7]:
df_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Financial_value,Older_Inactive,Low_value_Churner,Age_per_Tenure,Balance_per_Tenure,Products_per_Tenure,Points_per_Tenure,FinancialValue_per_Tenure,Engagement_per_Age,Balance_per_Products
0,619,France,Female,42,2,0.00,1,1,1,101348.88,...,101348.88,0,0,20.000000,0.000000,0.333333,154.666667,33782.960000,0.071429,0.000000
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,...,196350.44,0,0,37.272727,76188.963636,0.500000,228.000000,98175.220000,0.048780,83807.860000
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,...,273592.37,0,0,5.185185,19711.209877,0.333333,41.888889,30399.152222,0.095238,53220.266667
3,699,France,Female,39,1,0.00,2,0,0,93826.63,...,93826.63,0,0,35.454545,0.000000,1.000000,175.000000,46913.315000,0.051282,0.000000
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,...,204594.92,0,0,20.476190,59767.057143,0.333333,141.666667,68198.306667,0.069767,125510.820000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,...,96270.64,0,0,7.647059,0.000000,0.333333,50.000000,16045.106667,0.076923,0.000000
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,...,159069.38,0,0,3.465347,5680.159406,0.090909,70.090909,14460.852727,0.085714,57369.610000
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,...,42085.58,0,0,5.070423,0.000000,0.125000,70.500000,5260.697500,0.055556,0.000000
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,...,167963.83,0,0,13.548387,24217.841935,0.500000,84.750000,41990.957500,0.071429,37537.655000


<h3> 2. Split

Antes de começar efetivamente a construir o pré-processador, farei o split dos dados. Como já dito anteriormente, não há possibilidade de vazamento com as features criadas anteriormente.

In [8]:
X = df_train.drop(columns={"Churn"})
y = df_train["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

O parâmetro stratify=y foi usado para manter a mesma proporção de registros com churn e sem churn em cada dataset

In [9]:
#Tamanho dos datasets de treino e teste
print(f'O dataset de treino tem {X_train.shape[0]} linhas')
print(f'O dataset de teste tem {X_test.shape[0]} linhas')

O dataset de treino tem 8000 linhas
O dataset de teste tem 2000 linhas


In [10]:
#Proporção de churn em cada dataset
print(f'Proporcão de churn no dataset de treino')
print(f'{y_train.value_counts(normalize=True)}')
print(f'\nProporcão de churn no dataset de teste')
print(f'{y_test.value_counts(normalize=True)}')

Proporcão de churn no dataset de treino
Churn
0    0.79625
1    0.20375
Name: proportion, dtype: float64

Proporcão de churn no dataset de teste
Churn
0    0.796
1    0.204
Name: proportion, dtype: float64


<h3> 3. Pré-processamento

Abaixo irei codificar o pré-processador seguindo as técnicas que mencionei

In [11]:
#Colunas do One-Hot e do ordinal encoder
ohe_cols = ['Geography', 'Gender']
ordinal_cols = ['Card Type']

#Hierarquia do ordinal encoder
ordinal_categories = [['No card', 'SILVER', 'GOLD', 'DIAMOND', 'PLATINUM']]

print(f"As colunas utilizadas no One-Hot Encoder são: {ohe_cols}")
print(f"As colunas utilizadas no Ordinal Encoder são: {ordinal_cols}")

As colunas utilizadas no One-Hot Encoder são: ['Geography', 'Gender']
As colunas utilizadas no Ordinal Encoder são: ['Card Type']


In [12]:
#Pré-processador
preprocessor = ColumnTransformer([
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), ohe_cols),
    ('ordinal_encoder', OrdinalEncoder(categories=ordinal_categories), ordinal_cols)],
    remainder='passthrough'
)

In [13]:
#Utilizando o pré-processador no dataset de treino
X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_train_processed.shape

(8000, 26)

<h2> Treino e avaliação dos resultados

Nessa etapa do estudo, irei comparar 3 modelos diferentes baseados em árvores, com o intuito de verificar qual o melhor algoritmo para esse problema. Os modelos são: Decision Tree Classifier, Random Forest Classifier e Light GBM Classifier

Para a comparação, usarei uma técnica chamada validação cruzada K-fold estratificada (Stratified K-fold cross validation) para treinar cada modelo. A K-fold cross validation consiste em dividir um dataset em K subsets, mantendo a proporção de churn igual em cada um. Após isso, são feitas iterações onde K-1 subsets são utilizados para treinamento do modelo e o restante para teste, repetindo-se o processo até que todos os subconjuntos tenham sido usados como conjunto de avaliação

In [14]:
#Algoritmos utilizados
tree_models = {
              'Decision Tree': DecisionTreeClassifier(),
              'Random Forest': RandomForestClassifier(),
              'LightGBM': LGBMClassifier()
              }

In [15]:
#K-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for name, model in tree_models.items():
    #Validação cruzada
    scores = cross_val_score(
        model,
        X_train_processed,
        y_train,
        cv=skf,
        scoring='roc_auc',
        n_jobs=-1
    )

    model.fit(X_train_processed, y_train)

    y_train_proba = model.predict_proba(X_train_processed)[:, 1]
    train_score = roc_auc_score(y_train, y_train_proba)

    results.append({
        'Model': name,
        'ROC_AUC_Mean': scores.mean(),
        'Training_Score': train_score
    })

[LightGBM] [Info] Number of positive: 1630, number of negative: 6370
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000877 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2863
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203750 -> initscore=-1.363019
[LightGBM] [Info] Start training from score -1.363019


c:\Users\izaia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
#Transformando os resultados em um dataset
eval_df = pd.DataFrame(results)

eval_df

,Model,ROC_AUC_Mean,Training_Score
0,Decision Tree,0.674014,1.000000
1,Random Forest,0.850667,1.000000
2,LightGBM,0.853423,0.978764


Avaliação:
- Decision Tree: Apresentou forte sobreajuste, com desempenho perfeito no treino, mas baixo na validação. Em geral, isso indica que o modelo decorou os dados de treinamento e com isso não consegue generalizar bem para dados novos, mostrando ser um algoritmo com desempenho inferior.
- Random Forest: Embora tenha também tenha apresentado sobreajuste, assim como a Decision Tree, A Random Forest obteve um bom desempenho com médias de ROC-AUC próximas a 0,85.
- LightGBM: Dentre os três algoritmos, o LightGBM alcançou o melhor resultado. Além de ter o melhor ROC-AUC (acima de 0,85), também foi o modelo que teve a menor diferença nas médias de treino e validação, com apenas 0,15.

<h2> Conclusão